In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [2]:
# Load training data set
train = pd.read_json('../input/train.json')

In [3]:
print(train.head())

                                              band_1  \
0  [-27.878360999999998, -27.15416, -28.668615, -...   
1  [-12.242375, -14.920304999999999, -14.920363, ...   
2  [-24.603676, -24.603714, -24.871029, -23.15277...   
3  [-22.454607, -23.082819, -23.998013, -23.99805...   
4  [-26.006956, -23.164886, -23.164886, -26.89116...   

                                              band_2        id inc_angle  \
0  [-27.154118, -29.537888, -31.0306, -32.190483,...  dfd5f913   43.9239   
1  [-31.506321, -27.984554, -26.645678, -23.76760...  e25388fd   38.1562   
2  [-24.870956, -24.092632, -20.653963, -19.41104...  58b2aaa0   45.2859   
3  [-27.889421, -27.519794, -27.165262, -29.10350...  4cfc3a18   43.8306   
4  [-27.206915, -30.259186, -30.259186, -23.16495...  271f93f4   35.6256   

   is_iceberg  
0           0  
1           0  
2           1  
3           0  
4           0  


In [4]:
order_num = np.random.rand(train.shape[0])
train['order'] = order_num
train.sort_values('order', axis=0, inplace=True)
print(train.head())

                                                 band_1  \
838   [-25.806887, -24.088665, -25.280447, -27.95129...   
1151  [-24.989557, -23.797901, -22.750198, -25.51631...   
1275  [-27.474922, -27.12039, -27.844673, -28.230818...   
603   [-19.522421, -17.127726, -17.127726, -19.38574...   
718   [-25.653328, -23.374504, -23.60017, -24.566309...   

                                                 band_2        id inc_angle  \
838   [-25.280308, -23.654278, -23.654346, -25.02873...  1c46183a   35.2957   
1151  [-28.01491, -31.010216, -29.174866, -26.992035...  7a4a774f   37.6877   
1275  [-24.70887, -26.779724, -32.800362, -27.475048...  fcae243a   44.4635   
603   [-26.261272, -27.905064, -27.905064, -30.90036...  a59605a2   39.2337   
718   [-35.195755, -30.586819, -29.175243, -29.62081...  95be51e8   42.5591   

      is_iceberg     order  
838            1  0.000299  
1151           1  0.002923  
1275           0  0.003010  
603            1  0.003861  
718            1  0.00402

In [5]:
raw_y = train['is_iceberg'].tolist()
tensor_y = tf.one_hot(raw_y, depth=2)
with tf.Session() as sess:
    total_y = np.array(sess.run(tensor_y))
print(total_y.shape)

(1604, 2)


In [6]:
raw_x_1 = np.array(train['band_1'].tolist())
raw_x_2 = np.array(train['band_2'].tolist())
total_x = np.concatenate((raw_x_1, raw_x_2), axis = 1)
total_x = raw_x_1
print(total_x.shape)

(1604, 5625)


In [7]:
dev_x = total_x[0:300,:]
dev_y = total_y[0:300,:]
print(dev_x.shape)
train_x = total_x[300:,:]
train_y = total_y[300:,:]
print(train_x.shape)
print(train_y.shape)

(300, 5625)
(1304, 5625)
(1304, 2)


In [8]:
class MiniBatch(object):
    def __init__(self, x, y, batch_size):
        self.generator = self.slice_random(x, y, batch_size)
    
    def next_batch(self):
        try:
            return(next(self.generator))
        except StopIteration as e:
            return None, None
        
    @staticmethod    
    def slice_random(X, Y, mini_batch_size):
        tmp_X = X.copy()
        tmp_Y = Y.copy()
        cur_index = 0
        while(tmp_X.shape[0]>0):
            pick_size = tmp_X.shape[0] if tmp_X.shape[0]<mini_batch_size else mini_batch_size
            mini_batch_x = tmp_X[cur_index:cur_index + pick_size,:]
            mini_batch_y = tmp_Y[cur_index:cur_index + pick_size,:]
            cur_index += pick_size
            yield mini_batch_x, mini_batch_y

In [9]:
# test
test_mini_batch = MiniBatch(dev_x, dev_y, 50)


In [10]:
with tf.name_scope('reshape'):
    x_in = tf.placeholder(tf.float32)
    y_in = tf.placeholder(tf.float32)
    x = tf.reshape(x_in, [-1,75,75,1])
    y = tf.reshape(y_in, [-1, 2])

In [11]:
with tf.name_scope('forward'):
    W1 = tf.get_variable("W1", [5,5,1,5], initializer=tf.contrib.layers.xavier_initializer(seed=0))
    W2 = tf.get_variable("W2", [5,5,5,10], initializer=tf.contrib.layers.xavier_initializer(seed=0))
    Z1 = tf.nn.conv2d(x, W1, [1,1,1,1], 'SAME')
    # RELU
    A1 = tf.nn.relu(Z1)
    # MAXPOOL: window 5x5, sride 5, padding 'SAME'
    P1 = tf.nn.max_pool(A1, [1,5,5,1], [1,5,5,1], 'SAME')
    # CONV2D: filters W2, stride 1, padding 'SAME'
    Z2 = tf.nn.conv2d(P1, W2, [1,1,1,1], 'SAME')
    # RELU
    A2 = tf.nn.relu(Z2)
    # MAXPOOL: window 4x4, stride 4, padding 'SAME'
    P2 = tf.nn.max_pool(A2, [1,3,3,1], [1,3,3,1], 'SAME')
    # FLATTEN
    P2 = tf.contrib.layers.flatten(P2)
    # FULLY-CONNECTED without non-linear activation function (not not call softmax).
    # 6 neurons in output layer. Hint: one of the arguments should be "activation_fn=None" 
    
    F3 = tf.layers.dense(P2, 50, activation=tf.nn.relu)
    F4 = tf.layers.dropout(F3, rate=0.5)
    y_ = tf.contrib.layers.fully_connected(F4, 2, activation_fn=None)

In [12]:
with tf.name_scope('cost'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_, labels=y))

In [13]:
with tf.name_scope('optimizer'):
    optimizer = tf.train.AdamOptimizer().minimize(cost)

In [14]:
with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_,1))
    correct_prediction = tf.cast(correct_prediction, tf.float32)
    accuracy = tf.reduce_mean(correct_prediction)

In [15]:
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     tf.add_to_collection('accuracy', accuracy)
#     tf.add_to_collection('y_', y_)
#     tf.add_to_collection('x_in', x_in)
#     tf.add_to_collection('y_in', y_in)
#     # writer = tf.summary.FileWriter('./logs',sess.graph)
#     for i in range(6001):
#         minibatch = MiniBatch(train_x, train_y, 50)
#         x_batch,y_batch = minibatch.next_batch()
#         if x_batch is None or x_batch.shape[0]<30:
#             minibatch = MiniBatch(train_x, train_y, 50)
#             x_batch,y_batch = minibatch.next_batch()
#         # print('minibatch size {} and {}'.format(x_batch.shape, y_batch.shape))
#         if i%100 == 0:
#             train_accuracy = accuracy.eval(session=sess,
#                                             feed_dict={
#                                                 x_in:train_x, y_in:train_y
#                                             })
#             print('step {}, training accuracy {}'.format(i, train_accuracy))
#             print('test accuracy {}'.format(accuracy.eval(session=sess, feed_dict={x_in: dev_x , y_in: dev_y})))
#             saver = tf.train.Saver()
#             saved_path = saver.save(sess, './models/{}.ckpt'.format(i))
#             print('model saved to {}'.format(saved_path))
#         sess.run(optimizer, feed_dict={x_in:x_batch, y_in:y_batch})
#     # writer.close()
#     print('test accuracy {}'.format(accuracy.eval(session=sess, feed_dict={x_in: dev_x , y_in: dev_y})))
    

In [16]:
with tf.Session() as sess:
    saver = tf.train.import_meta_graph('./models-1/500.ckpt.meta')
    saver.restore(sess, './models-1/500.ckpt')
    accuracy = tf.get_collection('accuracy')[0]
    x_in = tf.get_collection('x_in')[0]
    y_in = tf.get_collection('y_in')[0]
    y_ = tf.get_collection('y_')[0]
    print('model restored')
    print('test accuracy {}'.format(accuracy.eval(session=sess, feed_dict={x_in: dev_x , y_in: dev_y})))
    res = y_.eval(session=sess, feed_dict={x_in: dev_x , y_in: dev_y})

ValueError: cannot add op with name W1/Adam as that name is already used

In [ ]:
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     writer = tf.summary.FileWriter('./logs',sess.graph)
#     sess.run(optimizer, feed_dict={x_in:train_x, y_in:train_y})
#     writer.close()

In [ ]:
res